In [1]:
import sys
sys.path.append('/fsx/matzeni/duck')

In [2]:
import torch
import h5py
import json
from pathlib import Path
import pickle
from tqdm import tqdm
import logging
from transformers import AutoTokenizer
import numpy as np
import faiss

from mblink.utils.utils import EntityCatalogue
from duck.datamodule import RelationCatalogue
from duck.common.utils import load_json
from duck.preprocessing.duck_index import DuckIndex
from duck.common.utils import load_json

logger = logging.getLogger()

In [3]:
D = torch.randint(4, (6000000, 1250), dtype=torch.uint8).cuda()

In [4]:
q = torch.randint(4, (1, 1250), dtype=torch.uint8).cuda()

In [55]:
%%time
batch_size = 1000000
results = []
for i in range(0, 6000000, batch_size):
    Q = q.expand((batch_size, 1250))
    R = D[i:i+batch_size].bitwise_xor(Q).count_nonzero(dim=-1)
    topk = torch.topk(R, k=100, largest=False)
    results.append(topk)

CPU times: user 820 µs, sys: 84 µs, total: 904 µs
Wall time: 910 µs


In [59]:
import math
math.ceil(5.5)

6

In [60]:
res = faiss.StandardGpuResources()
index = faiss.IndexBinaryFlat(128)
index1 = faiss.GpuIndexBinaryFlat(res, index)
index2 = faiss.GpuIndexBinaryFlat(res, index)

In [63]:
x = np.eye(10)
x

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [73]:
sort_permutation = np.argsort(x, axis=-1)

In [74]:
for i in range(10):
    print(x[i][sort_permutation[i]])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [75]:
i = 3
f"shard{i:02d}.faiss"

'shard03'

In [76]:
faiss.Index

faiss.swigfaiss.Index

In [86]:
list(Path("/fsx/matzeni/data/duck").glob("p*.json"))

[PosixPath('/fsx/matzeni/data/duck/properties.json')]